# Notebook 04: Combined Statistical Analysis

**Goal:** Combine all previous analyses and run statistical tests

**Combines:**
- Notebook 01: Initial cushion measurements
- Notebook 02: Post-throw erosion patterns (optional)
- Notebook 03: Pre-throw growth patterns

**Key Questions:**
1. Does pre-throw strategy affect EPA?
2. Does post-throw execution affect EPA?
3. What's the optimal CB coverage approach?

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os

print("=" * 70)
print("NOTEBOOK 04: COMBINED STATISTICAL ANALYSIS")
print("=" * 70)

# Load datasets
print("\nLoading datasets...")

# Required: Cushion data
cushion_df = pd.read_csv('../data/processed/cushion_analysis_data.csv')
print(f"  Cushion data: {len(cushion_df):,} rows")

# Optional: Erosion data
erosion_df = None
try:
    erosion_df = pd.read_csv('../data/processed/erosion_analysis_data.csv')
    print(f"  Erosion data: {len(erosion_df):,} rows")
    print(f"    Columns: {list(erosion_df.columns)}")
except FileNotFoundError:
    print("  Erosion data: NOT FOUND (Notebook 02 not complete - continuing without it)")

# Required: Pre-throw data
prethrow_df = None
try:
    prethrow_df = pd.read_csv('../data/processed/prethrow_coverage_data.csv')
    print(f"  Pre-throw data: {len(prethrow_df):,} rows")
except FileNotFoundError:
    print("  Pre-throw data: NOT FOUND")
    print("\n  ERROR: You must run Notebook 03 first!")
    raise

# Merge datasets
print("\nMerging datasets...")
merge_keys = ['game_id', 'play_id', 'wr_nfl_id', 'cb_nfl_id']

combined_df = cushion_df.copy()

# Merge erosion data (if available)
if erosion_df is not None:
    # Only use columns that actually exist
    available_cols = [col for col in ['erosion_rate', 'final_cushion'] if col in erosion_df.columns]
    if available_cols:
        erosion_subset = erosion_df[merge_keys + available_cols]
        combined_df = combined_df.merge(erosion_subset, on=merge_keys, how='left')
        print(f"  After erosion merge: {len(combined_df):,} rows")
        print(f"    Added columns: {available_cols}")
    else:
        print("  Skipping erosion merge (no compatible columns)")

# Merge pre-throw data (required)
if prethrow_df is not None:
    prethrow_cols = ['growth_rate', 'prethrow_pattern', 'cushion_growth', 'presnap_cushion', 'at_throw_cushion']
    available_prethrow_cols = [col for col in prethrow_cols if col in prethrow_df.columns]
    prethrow_subset = prethrow_df[merge_keys + available_prethrow_cols]
    combined_df = combined_df.merge(prethrow_subset, on=merge_keys, how='left')
    print(f"  After pre-throw merge: {len(combined_df):,} rows")
    print(f"    Added columns: {available_prethrow_cols}")

# Drop rows with missing key metrics
before_drop = len(combined_df)
combined_df = combined_df.dropna(subset=['epa', 'growth_rate']).copy()
print(f"\n✓ Final dataset: {len(combined_df):,} complete pairings")
print(f"  (Dropped {before_drop - len(combined_df):,} incomplete rows)")

# Statistical tests
print("\n" + "=" * 70)
print("STATISTICAL TESTS")
print("=" * 70)

# Test 1: Pre-throw pattern vs EPA
print("\n1. Does pre-throw strategy affect EPA?")
if 'prethrow_pattern' in combined_df.columns:
    pattern_groups = combined_df.groupby('prethrow_pattern')['epa'].apply(list)
    if len(pattern_groups) >= 2:
        f_stat, p_value = stats.f_oneway(*pattern_groups)
        print(f"   ANOVA: F={f_stat:.3f}, p={p_value:.4f}")
        if p_value < 0.05:
            print("   → YES! Pre-throw strategy significantly affects EPA (p < 0.05)")
        else:
            print("   → No significant effect found (p >= 0.05)")
        
        print("\n   EPA by Pre-Throw Pattern:")
        for pattern in sorted(combined_df['prethrow_pattern'].unique()):
            pattern_epa = combined_df[combined_df['prethrow_pattern'] == pattern]['epa']
            print(f"     {pattern:25} Mean EPA: {pattern_epa.mean():+.3f} (n={len(pattern_epa)})")
else:
    print("   Pattern data not available")

# Test 2: Correlation between growth_rate and EPA
print("\n2. Correlation: Pre-throw growth rate vs EPA")
correlation = combined_df['growth_rate'].corr(combined_df['epa'])
print(f"   Pearson r = {correlation:.4f}")
if abs(correlation) > 0.1:
    direction = 'Positive' if correlation > 0 else 'Negative'
    strength = 'Strong' if abs(correlation) > 0.3 else 'Moderate' if abs(correlation) > 0.2 else 'Weak'
    print(f"   → {strength} {direction} correlation detected")
else:
    print("   → Negligible correlation")

# Test 3: Route-specific analysis
print("\n3. Pre-throw strategy by route type:")
for route in ['HITCH', 'OUT', 'SLANT', 'GO', 'CROSS']:
    route_df = combined_df[combined_df['route'] == route]
    if len(route_df) >= 20:
        print(f"\n   {route} (n={len(route_df)}):")
        print(f"     Avg growth rate: {route_df['growth_rate'].mean():+.3f} y/frame")
        print(f"     Avg EPA:         {route_df['epa'].mean():+.3f}")

# Create visualization
print("\n" + "=" * 70)
print("CREATING VISUALIZATION")
print("=" * 70)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Panel 1: Growth rate vs EPA scatter
ax1.scatter(combined_df['growth_rate'], combined_df['epa'], alpha=0.3, s=20, color='steelblue')
ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5, label='Neutral EPA')
ax1.axvline(x=0, color='blue', linestyle='--', alpha=0.5, label='No growth')
ax1.set_xlabel('Pre-Throw Growth Rate (y/frame)', fontsize=11)
ax1.set_ylabel('EPA', fontsize=11)
ax1.set_title(f'Pre-Throw Strategy vs EPA\n(r={correlation:.3f})', fontsize=12, weight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Panel 2: Pre-throw pattern distribution
if 'prethrow_pattern' in combined_df.columns:
    pattern_counts = combined_df['prethrow_pattern'].value_counts()
    colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(pattern_counts)))
    ax2.barh(range(len(pattern_counts)), pattern_counts.values, color=colors)
    ax2.set_yticks(range(len(pattern_counts)))
    ax2.set_yticklabels(pattern_counts.index)
    ax2.set_xlabel('Count', fontsize=11)
    ax2.set_title('Pre-Throw Pattern Distribution', fontsize=12, weight='bold')
    ax2.grid(True, axis='x', alpha=0.3)
else:
    ax2.text(0.5, 0.5, 'Pattern data not available', ha='center', va='center')
    ax2.axis('off')

# Panel 3: EPA by pattern
if 'prethrow_pattern' in combined_df.columns:
    pattern_epa = combined_df.groupby('prethrow_pattern')['epa'].mean().sort_values()
    colors = ['green' if x > 0 else 'red' for x in pattern_epa.values]
    ax3.barh(range(len(pattern_epa)), pattern_epa.values, color=colors, alpha=0.7)
    ax3.set_yticks(range(len(pattern_epa)))
    ax3.set_yticklabels(pattern_epa.index)
    ax3.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax3.set_xlabel('Average EPA', fontsize=11)
    ax3.set_title('Average EPA by Pre-Throw Pattern', fontsize=12, weight='bold')
    ax3.grid(True, axis='x', alpha=0.3)
else:
    ax3.text(0.5, 0.5, 'Pattern data not available', ha='center', va='center')
    ax3.axis('off')

# Panel 4: Erosion vs EPA (if available)
if 'erosion_rate' in combined_df.columns:
    valid_erosion = combined_df.dropna(subset=['erosion_rate'])
    ax4.scatter(valid_erosion['erosion_rate'], valid_erosion['epa'], alpha=0.3, s=20, color='coral')
    ax4.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    ax4.axvline(x=0, color='blue', linestyle='--', alpha=0.5)
    ax4.set_xlabel('Post-Throw Erosion Rate (y/frame)', fontsize=11)
    ax4.set_ylabel('EPA', fontsize=11)
    ax4.set_title('Post-Throw Execution vs EPA', fontsize=12, weight='bold')
    ax4.grid(True, alpha=0.3)
else:
    ax4.text(0.5, 0.5, 'Erosion data not available\n(Optional - Run Notebook 02)', 
             ha='center', va='center', fontsize=11)
    ax4.axis('off')

plt.suptitle('Combined Coverage Analysis: Pre-Throw Strategy & EPA', 
             fontsize=14, weight='bold', y=0.995)
plt.tight_layout()

# Save visualization
os.makedirs('../visualizations', exist_ok=True)
plt.savefig('../visualizations/combined_analysis_summary.png', dpi=150, bbox_inches='tight')
print("\n✓ Visualization saved: ../visualizations/combined_analysis_summary.png")
plt.close()

# Save combined dataset
output_path = '../data/processed/final_combined_analysis.csv'
combined_df.to_csv(output_path, index=False)
print(f"\n✓ Combined dataset saved: {output_path}")
print(f"  Rows: {len(combined_df):,}")
print(f"  Columns: {len(combined_df.columns)}")

# Summary
print("\n" + "=" * 70)
print("ANALYSIS COMPLETE")
print("=" * 70)
print(f"\n📊 Dataset Summary:")
print(f"  • Total pairings: {len(combined_df):,}")
print(f"  • Unique games: {combined_df['game_id'].nunique()}")
print(f"  • Unique routes: {combined_df['route'].nunique()}")
print(f"  • Average EPA: {combined_df['epa'].mean():+.3f}")
print(f"  • Average growth rate: {combined_df['growth_rate'].mean():+.3f} y/frame")

if 'prethrow_pattern' in combined_df.columns:
    pattern_epa = combined_df.groupby('prethrow_pattern')['epa'].mean().sort_values(ascending=False)
    print(f"\n🎯 Key Findings:")
    print(f"  • Best strategy: {pattern_epa.index[0]}")
    print(f"    (Average EPA: {pattern_epa.iloc[0]:+.3f})")
    print(f"  • Worst strategy: {pattern_epa.index[-1]}")
    print(f"    (Average EPA: {pattern_epa.iloc[-1]:+.3f})")
    print(f"  • EPA difference: {pattern_epa.iloc[0] - pattern_epa.iloc[-1]:.3f}")

print(f"\n💾 Files created:")
print(f"  • {output_path}")
print(f"  • ../visualizations/combined_analysis_summary.png")
print("\n" + "=" * 70)
print("✓ Analysis pipeline complete!")
print("=" * 70)

NOTEBOOK 04: COMBINED STATISTICAL ANALYSIS

Loading datasets...
  Cushion data: 1,439 rows
  Erosion data: 1,439 rows
    Columns: ['game_id', 'play_id', 'week', 'wr_nfl_id', 'cb_nfl_id', 'wr_name', 'cb_name', 'wr_x', 'wr_y', 'cb_x', 'cb_y', 'initial_cushion', 'ball_land_x', 'ball_land_y', 'y_dist', 'x_dist', 'route', 'epa', 'yards_gained', 'team_coverage_type', 'down', 'yards_to_go', 'receiver_alignment', 'initial_cushion_check', 'final_cushion', 'num_frames', 'total_erosion', 'erosion_rate', 'pct_maintained', 'pattern']
  Pre-throw data: 1,439 rows

Merging datasets...
  After erosion merge: 1,439 rows
    Added columns: ['erosion_rate', 'final_cushion']
  After pre-throw merge: 1,439 rows
    Added columns: ['growth_rate', 'prethrow_pattern', 'cushion_growth', 'presnap_cushion', 'at_throw_cushion']

✓ Final dataset: 1,439 complete pairings
  (Dropped 0 incomplete rows)

STATISTICAL TESTS

1. Does pre-throw strategy affect EPA?
   ANOVA: F=3.074, p=0.0156
   → YES! Pre-throw strategy